In [1]:
# Import Libraries and Dependencies
import pandas as pd

### 1. Combine and Clean the Data
#### Import CSVs

In [3]:
# Read the CSV files into DataFrames.
# File paths
file_2020 = 'Resources/athletic_sales_2020.csv'
file_2021 = 'Resources/athletic_sales_2021.csv'

# Read the CSV files into DataFrames
sales_2020 = pd.read_csv(file_2020)
sales_2021 = pd.read_csv(file_2021)

In [4]:
# Display the 2020 sales DataFrame
sales_2020.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000.00,In-store
1,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,68,83,5644,2426.92,Online
2,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,75,275,206250,61875.00,Outlet
3,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,34,384,13056,6789.12,Outlet
4,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,53,83,4399,1407.68,Outlet


In [5]:
# Display the 2021 sales DataFrame
sales_2021.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,65,750,487500,121875.00,Outlet
1,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,51,233,11883,3208.41,Outlet
2,Kohl's,1189833,1/1/21,Midwest,Montana,Billings,Men's Apparel,50,275,137500,82500.00,Outlet
3,Kohl's,1189833,1/1/21,Midwest,Montana,Billings,Men's Apparel,47,77,3619,2714.25,Online
4,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,64,225,14400,5184.00,Online


#### Check the data types of each DataFrame

In [6]:
# Check the 2020 sales data types.
sales_2020.dtypes


retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object

In [7]:
# Check the 2021 sales data types.
sales_2021.dtypes


retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object

#### Combine the sales data by rows.

In [9]:
# Combine the 2020 and 2021 sales DataFrames on the rows and reset the index
combined_sales = pd.concat([sales_2020, sales_2021], axis=0).reset_index(drop=True)
combined_sales.head()


,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000.00,In-store
1,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,68,83,5644,2426.92,Online
2,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,75,275,206250,61875.00,Outlet
3,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,34,384,13056,6789.12,Outlet
4,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,53,83,4399,1407.68,Outlet


In [10]:
# Check if any values are null.
combined_sales.isnull().sum()


retailer            0
retailer_id         0
invoice_date        0
region              0
state               0
city                0
product             0
price_per_unit      0
units_sold          0
total_sales         0
operating_profit    0
sales_method        0
dtype: int64

In [11]:
# Check the data type of each column
combined_sales.dtypes


retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object

In [13]:
# Convert the "invoice_date" to a datetime datatype
combined_sales['invoice_date'] = pd.to_datetime(combined_sales['invoice_date'], format='%m/%d/%y')


In [14]:
# Confirm that the "invoice_date" data type has been changed
combined_sales['invoice_date'].dtype


dtype('<M8[ns]')

### 2. Determine which Region Sold the Most Products

#### Using `groupby`

In [18]:
# Show the number products sold for region, state, and city.
# Rename the sum to "Total_Products_Sold".
region_sales = combined_sales.groupby('region')['units_sold'].sum().rename('Total_Products_Sold')

# Show the top 5 results
top_regions = region_sales.nlargest(5)
top_regions
                            

region
West         685563
Northeast    501279
South        492260
Southeast    407000
Midwest      390914
Name: Total_Products_Sold, dtype: int64

#### Using `pivot_table`

In [19]:
# Group by region, state, and city and sum the units_sold
region_state_city_sales_pivot = combined_sales.pivot_table(
    index=['region', 'state', 'city'],
    values='units_sold',
    aggfunc='sum'
).reset_index()

# Rename the 'units_sold' column to 'Total_Products_Sold'
region_state_city_sales_pivot.rename(columns={'units_sold': 'Total_Products_Sold'}, inplace=True)

# Sort and show the top 5 results based on 'Total_Products_Sold'
top_regions_states_cities_pivot = region_state_city_sales_pivot.sort_values('Total_Products_Sold', ascending=False).head(5)

top_regions_states_cities_pivot

,region,state,city,Total_Products_Sold
21,Northeast,New York,New York,111954
33,South,Texas,Houston,90322
44,West,California,San Francisco,85478
43,West,California,Los Angeles,76384
34,Southeast,Florida,Miami,73135


### 3. Determine which Region had the Most Sales

#### Using `groupby`

In [21]:
# Show the total sales for the products sold for each region, state, and city.
region_sales_total = combined_sales.groupby(['region', 'state', 'city'])['total_sales'].sum().reset_index()

# Rename the summed column to 'Total_Sales'
region_sales_total.rename(columns={'total_sales': 'Total_Sales'}, inplace=True)


# Show the top 5 results.
top_regions_by_sales = region_sales_total.sort_values('Total_Sales', ascending=False).head(5)

top_regions_by_sales

,region,state,city,Total_Sales
21,Northeast,New York,New York,39801235
44,West,California,San Francisco,33973228
34,Southeast,Florida,Miami,31600863
39,Southeast,South Carolina,Charleston,29285637
35,Southeast,Florida,Orlando,27682851


#### Using `pivot_table`

In [22]:
# Show the total sales for the products sold for each region, state, and city.
region_sales_pivot = combined_sales.pivot_table(
    index=['region', 'state', 'city'],
    values='total_sales',
    aggfunc='sum'
).reset_index()


# Optional: Rename the "total_sales" column to "Total Sales"
region_sales_pivot.rename(columns={'total_sales': 'Total_Sales'}, inplace=True)


# Show the top 5 results.
top_regions_by_sales_pivot = region_sales_pivot.sort_values('Total_Sales', ascending=False).head(5)

top_regions_by_sales_pivot

,region,state,city,Total_Sales
21,Northeast,New York,New York,39801235
44,West,California,San Francisco,33973228
34,Southeast,Florida,Miami,31600863
39,Southeast,South Carolina,Charleston,29285637
35,Southeast,Florida,Orlando,27682851


### 4. Determine which Retailer had the Most Sales

#### Using `groupby`

In [24]:
# Show the total sales for the products sold for each retailer, region, state, and city.
retailer_sales_groupby = combined_sales.groupby(['retailer', 'region', 'state', 'city'])['total_sales'].sum().reset_index()

# Rename the 'total_sales' column to 'Total_Sales'
retailer_sales_groupby.rename(columns={'total_sales': 'Total_Sales'}, inplace=True)

# Show the top 5 results.
top_retailers_by_sales_groupby = retailer_sales_groupby.sort_values('Total_Sales', ascending=False).head(5)

top_retailers_by_sales_groupby

,retailer,region,state,city,Total_Sales
103,West Gear,West,California,San Francisco,32794405
50,Kohl's,West,California,Los Angeles,25127160
22,Foot Locker,Northeast,New York,New York,25008568
109,West Gear,West,Washington,Seattle,24862675
33,Foot Locker,Southeast,South Carolina,Charleston,24822280


#### Using `pivot_table`

In [25]:
# Show the total sales for the products sold for each retailer, region, state, and city.
retailer_sales_pivot = combined_sales.pivot_table(
    index=['retailer', 'region', 'state', 'city'],
    values='total_sales',
    aggfunc='sum'
).reset_index()

# Optional: Rename the "total_sales" column to "Total Sales"
retailer_sales_pivot.rename(columns={'total_sales': 'Total_Sales'}, inplace=True)

# Show the top 5 results.
top_retailers_by_sales_pivot = retailer_sales_pivot.sort_values('Total_Sales', ascending=False).head(5)

top_retailers_by_sales_pivot

,retailer,region,state,city,Total_Sales
103,West Gear,West,California,San Francisco,32794405
50,Kohl's,West,California,Los Angeles,25127160
22,Foot Locker,Northeast,New York,New York,25008568
109,West Gear,West,Washington,Seattle,24862675
33,Foot Locker,Southeast,South Carolina,Charleston,24822280


### 5. Determine which Retailer Sold the Most Women's Athletic Footwear

In [26]:
# Filter the sales data to get the women's athletic footwear sales data.
womens_footwear_sales = combined_sales[combined_sales['product'].str.contains("Women's") & combined_sales['product'].str.contains("Footwear")]


#### Using `groupby`

In [27]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
womens_footwear_sales_groupby = womens_footwear_sales.groupby(['retailer', 'region', 'state', 'city'])['units_sold'].sum().reset_index()

# Rename the 'units_sold' column to 'Womens_Footwear_Units_Sold'
womens_footwear_sales_groupby.rename(columns={'units_sold': 'Womens_Footwear_Units_Sold'}, inplace=True)

# Show the top 5 results.
top_retailers_womens_footwear_groupby = womens_footwear_sales_groupby.sort_values('Womens_Footwear_Units_Sold', ascending=False).head(5)

top_retailers_womens_footwear_groupby

,retailer,region,state,city,Womens_Footwear_Units_Sold
103,West Gear,West,California,San Francisco,25453
22,Foot Locker,Northeast,New York,New York,23084
50,Kohl's,West,California,Los Angeles,22856
69,Sports Direct,South,Texas,Dallas,20653
83,Walmart,South,Texas,Houston,20173


#### Using `pivot_table`

In [28]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
womens_footwear_sales_pivot = womens_footwear_sales.pivot_table(
    index=['retailer', 'region', 'state', 'city'],
    values='units_sold',
    aggfunc='sum'
).reset_index()

# Rename the 'units_sold' column to 'Womens_Footwear_Units_Sold'
womens_footwear_sales_pivot.rename(columns={'units_sold': 'Womens_Footwear_Units_Sold'}, inplace=True)

# Show the top 5 results.
top_retailers_womens_footwear_pivot = womens_footwear_sales_pivot.sort_values('Womens_Footwear_Units_Sold', ascending=False).head(5)

top_retailers_womens_footwear_pivot

,retailer,region,state,city,Womens_Footwear_Units_Sold
103,West Gear,West,California,San Francisco,25453
22,Foot Locker,Northeast,New York,New York,23084
50,Kohl's,West,California,Los Angeles,22856
69,Sports Direct,South,Texas,Dallas,20653
83,Walmart,South,Texas,Houston,20173


### 5. Determine the Day with the Most Women's Athletic Footwear Sales

In [49]:
# Create a pivot table with the 'invoice_date' column is the index, and the "total_sales" as the values.
sales_by_date = combined_sales.pivot_table(
    index='invoice_date',
    values='total_sales'
)


# Optional: Rename the "total_sales" column to "Total Sales"
sales_by_date.rename(columns={'total_sales': 'Total_Sales'}, inplace=True)


# Show the table.
sales_by_date


,Total_Sales
invoice_date,
2020-01-01,140856.833333
2020-01-02,114901.666667
2020-01-03,105428.833333
2020-01-04,102513.333333
2020-01-05,117971.500000
...,...
2021-12-27,120470.600000
2021-12-28,60202.000000
2021-12-29,42221.800000


In [52]:
# Resample the pivot table into daily bins, and get the total sales for each day.
daily_sales = sales_by_date.resample('D').sum()

# Add a new column 'Day_of_Week' to store the day of the week corresponding to the dates
daily_sales['Day_of_Week'] = daily_sales.index.day_name()

# Sort the resampled pivot table in ascending order on "Total Sales".
daily_sales.sort_values('Total_Sales', ascending=False)



,Total_Sales,Day_of_Week
invoice_date,,
2020-09-17,374745.0,Thursday
2020-06-30,346418.0,Tuesday
2020-07-01,338723.5,Wednesday
2020-07-07,338055.0,Tuesday
2020-07-08,308085.0,Wednesday
...,...,...
2020-10-13,0.0,Tuesday
2020-10-12,0.0,Monday
2020-10-11,0.0,Sunday


### 6.  Determine the Week with the Most Women's Athletic Footwear Sales

In [54]:
# Resample the pivot table into weekly bins, and get the total sales for each week.
weekly_sales = sales_by_date.resample('W').sum()

# Add a new column 'Week_of_Year' to store the week of the year corresponding to the dates
weekly_sales['Week_of_Year'] = weekly_sales.index.strftime('%U')

# Sort the resampled pivot table in descending order on "Total Sales".
weekly_sales.sort_values('Total_Sales', ascending=False)


,Total_Sales,Week_of_Year
invoice_date,,
2020-07-05,1.880870e+06,27
2020-07-12,1.787918e+06,28
2020-10-04,1.627700e+06,40
2020-09-20,1.602208e+06,38
2020-09-27,1.557548e+06,39
...,...,...
2022-01-02,2.962068e+05,01
2020-12-20,2.951135e+05,51
2021-10-31,2.654317e+05,44
